In [1]:
import math, torch, gc
from transformers import (
    AutoTokenizer,
    Trainer,
    TrainingArguments,
    default_data_collator
)
from torch.utils.data import DataLoader
from transformers import AutoModelForCausalLM
# from transformers.models.gpt2.modeling_gpt2 import GPT2LMHeadModel
# from transformers.models.gpt2.modeling_gpt2 import GemmaForCausalLM

/home/ubuntu/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-07-27 12:31:34.526464: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-27 12:31:35.325172: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753619495.643989    2065 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753619495.738492    2065 cuda_blas.cc:1407] Unable to 

In [2]:
import os
import sys

# change working directory
current_dir = os.getcwd() 
print(f"Current Working Directory: {current_dir}")
parent_dir = os.path.abspath(os.path.join(current_dir, '..'))
os.chdir(parent_dir)
current_dir = os.getcwd()
print(f"Current Working Directory: {current_dir}")
#We need to be in the main directory that contains tests, models, etc folders

Current Working Directory: /lambda/nfs/codebase2/tests
Current Working Directory: /lambda/nfs/codebase2


In [3]:
#----------------- load training data ---------------------------------------
import random
from datasets import load_from_disk
lm_ds = load_from_disk("./data/gemma_300MB_bs512")

print("original rows:", len(lm_ds))          # e.g. 50 092

original rows: 136244


In [4]:
# choose how many you want to keep
KEEP = 15_000                         # keep 10 k blocks
SEED = 42
random.seed(SEED)

# draw KEEP unique indices and select them
idx = random.sample(range(len(lm_ds)), KEEP) # random sample, no replacement
lm_ds_small = lm_ds.select(idx)              # constant-time view
print("trimmed rows :", len(lm_ds_small))    # 10 000

trimmed rows : 15000


In [5]:
# ─── hyper-parameters ────────────────────────────────────────────
MODEL_NAME = "gemma2b-it"
MODEL_PATH="./models/"+MODEL_NAME
SPAN       = 124           # sliding-window width
BATCH      = 8
EPOCHS     = 2
LR         = 5e-7
SEED       = 42
DEVICE     = "cuda"

torch.manual_seed(SEED)

# ─── 1) sliding-window mask builder ──────────────────────────────
# from coarse_grain_model import GemmaWithSlidingWindow

In [6]:
# ─── 3) load tokenizer & dataset ─────────────────────────────────
tok = AutoTokenizer.from_pretrained(MODEL_PATH, use_fast=True)
tok.pad_token = tok.eos_token
tok.padding_side = "left"

In [7]:
# ─── 4) load & wrap model ────────────────────────────────────────
model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    torch_dtype=torch.bfloat16
).to(DEVICE)

Loading checkpoint shards: 100%|██████████| 3/3 [00:04<00:00,  1.43s/it]


In [8]:
# ─── 5) define collator that adds labels ──────────────────────────
def causal_collator_with_labels(features):
    batch = default_data_collator(features)
    batch["labels"] = batch["input_ids"].clone()
    return batch

# ─── 6) prepare Trainer ──────────────────────────────────────────
args = TrainingArguments(
    output_dir                  = "./models/"+MODEL_NAME+"_nomask2", #"_sw"+str(SPAN),
    overwrite_output_dir        = True,
    num_train_epochs            = EPOCHS,
    per_device_train_batch_size = BATCH,
    fp16                        = False,             # FP16 activations + kernel fusions
    bf16                        = True,
    gradient_checkpointing      = False,            # disable re-compute
    learning_rate               = LR,
    warmup_ratio                = 0.1,
    optim                       = "paged_adamw_32bit",
    logging_steps               = 500,
    save_strategy               = "epoch",
    report_to                   = "none",
    seed                        = SEED,
    remove_unused_columns       = False,
    dataloader_num_workers      = 4,                # parallel data loading
    # pin_memory + prefetch inside Trainer by default
)

trainer = Trainer(
    model          = model,
    args           = args,
    train_dataset  = lm_ds_small,
    eval_dataset   = None,
    data_collator  = causal_collator_with_labels,
    tokenizer      = tok,
)

/tmp/ipykernel_2065/1944533246.py:28: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [9]:
# ─── 7) train & evaluate ─────────────────────────────────────────
print(f"🚀 Training Gemma with sliding-window b={SPAN} …")
trainer.train()

with torch.no_grad():
    eval_loss = trainer.evaluate(lm_ds_small.select(range(1024)))["eval_loss"]
    print(f"✓ final PPL ≈ {math.exp(eval_loss):.2f}")

# ─── 8) cleanup ─────────────────────────────────────────────────
del model, trainer
torch.cuda.empty_cache()
gc.collect()

🚀 Training Gemma with sliding-window b=124 …


Step,Training Loss
500,4.560300
1000,3.436400
1500,3.354200
2000,3.344200
2500,3.329100
3000,3.337900
3500,3.328600


✓ final PPL ≈ 28.33


3976